In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from scipy.sparse import csr_matrix, hstack, lil_matrix
from sklearn.preprocessing import MinMaxScaler

In [2]:
interactions = pd.read_csv('Sample data/train/behaviors.tsv', sep='\t',header=None, index_col=0, usecols=[0, 1, 3, 4])
interactions.columns = ['uID','history','impLog']

interactions['impLog'] = interactions['impLog'].apply(lambda x: [(y.split('-')[0], int(y.split('-')[1])) for y in x.split(' ')])
interactions['history'] = interactions['history'].str.split()

In [3]:
news = pd.read_csv('Sample data/train/news.tsv',sep='\t',header=None, index_col=0)
news.columns = ["Category", "SubCategory", "Title", "Abstract", "URL", "Entities", "RelatedEntities"]
news.drop(columns=["URL", "Entities", "RelatedEntities"], inplace=True)

In [4]:
# tf-idf vectorization
def vectorize(data):
    vectorizer = TfidfVectorizer(max_features=1000)
    vectorizer.fit(data)
    return vectorizer.transform(data)
tfidf = vectorize(news['Title'])
# one-hot encoding
def one_hot(data):
    enc = OneHotEncoder()
    return enc.fit_transform(data)
one_hot_category = one_hot(news[['Category']])
one_hot_subcategory = one_hot(news[['SubCategory']])
# combine tf-idf and one_hot_category and one_hot_subcategory
news_vector = hstack([tfidf, one_hot_category, one_hot_subcategory])
news_map = dict(zip(news.index, news_vector.toarray()))


In [5]:
temp = interactions.copy()
temp['impLog'] = temp['impLog'].apply(lambda x: [tup[0] for tup in x if tup[1] == 1])

def concatenate_logs(history, impLog):
    if history is np.NaN and history is np.NaN:
        return []
    elif history is np.NaN:
        return impLog
    elif history is np.NaN:
        return history
    else:
        return history + impLog

# create a userDict where key are all unique user IDs and value is a list of read newsID (all newsIDs in history + newsIds in impLog where label is 1)
user_dict = {row['uID']: concatenate_logs(row['history'], row['impLog']) for _, row in temp.iterrows()}

In [6]:
# news_ids = {}
# for idx, nID in enumerate(set(news.index)):
#     news_ids[nID] = idx

# user_ids = {}
# for idx, uID in enumerate(set(interactions['uID'])):
#     user_ids[uID] = idx

# # create an embedding matrix of shape (n_unique_users, n_unique_news)
# embed = lil_matrix((len(user_ids), len(news_ids))) 
# for u in user_dict:
#     for n in user_dict[u]:
#         embed[user_ids[u], news_ids[n]] = 1

In [19]:
# from sklearn.decomposition import TruncatedSVD

# svd = TruncatedSVD(n_components=50, random_state=42)
# embed_svd = svd.fit_transform(embed)

In [8]:
# label/sub-label user-hitory one-hot encoding

def process_row(row, categories):
    category_count = {category: 0 for category in categories}
    for category in row:
        if category in category_count:
            category_count[category] += 1
    category_count = {k: category_count[k] for k in sorted(category_count)}
    return list(category_count.values())

def labels_one_hot(category):
    categories = news[category].unique()
    interactions[category] = interactions['history'].apply(lambda x: [news.loc[nID][category] for nID in x] if x is not np.NaN else [])
    return interactions[category].apply(lambda x: process_row(x, categories))

interactions['category_hist_encoded'] = labels_one_hot('Category')
interactions['subcategory_hist_encoded'] = labels_one_hot('SubCategory')

In [20]:
interactions_explode = interactions[['uID', 'impLog', 'category_hist_encoded', 'subcategory_hist_encoded']].explode('impLog')

interactions_explode['nID'] = interactions_explode['impLog'].apply(lambda x: news_map[x[0]] if x[0] in news_map else [0] * len(list(news_map.values())[0]))
interactions_explode['label'] = interactions_explode['impLog'].apply(lambda x: x[1])
# interactions_explode['uID'] = interactions_explode['uID'].apply(lambda x: embed_svd[user_ids[x], :])

In [10]:
# user_id one-hot encoding
# encoder = OneHotEncoder()
# user_encoded = encoder.fit_transform(interactions_explode[['uID']])

# standardized user history 
scalar = MinMaxScaler()
category_hist_encoded = scalar.fit_transform(interactions_explode['category_hist_encoded'].to_list())
subcategory_hist_encoded = scalar.fit_transform(interactions_explode['subcategory_hist_encoded'].to_list())

In [11]:
news_encoded = csr_matrix(interactions_explode['nID'].to_list())

In [21]:
# embed_encoded = csr_matrix(interactions_explode['uID'].to_list())

In [27]:
# X_train = hstack([category_hist_encoded, subcategory_hist_encoded, news_encoded, embed_encoded]).tocsr()
X_train = hstack([category_hist_encoded, subcategory_hist_encoded, news_encoded]).tocsr()

In [14]:
y_train = interactions_explode['label'].to_numpy()

In [15]:
from numba import njit
from tqdm import trange
from sklearn.base import BaseEstimator, ClassifierMixin


class FactorizationMachineClassifier(BaseEstimator, ClassifierMixin):
    """
    Factorization Machine [1]_ using Stochastic Gradient Descent.
    For binary classification only.

    Parameters
    ----------
    n_iter : int, default 10
        Number of iterations to train the algorithm.

    n_factors : int, default 10
        Number/dimension of features' latent factors.

    learning_rate : float, default 0.1
        Learning rate for the gradient descent optimizer.

    reg_coef : float, default 0.01
        Regularization strength for weights/coefficients.

    reg_factors : float, default 0.01
        Regularization strength for features' latent factors.

    random_state : int, default 1234
        Seed for the randomly initialized features latent factors

    verbose : bool, default True
        Whether to print progress bar while training.

    Attributes
    ----------
    intercept_ : double
        Intercept term, w0 based on the original notations.

    coef_ : 1d ndarray, shape [n_features,]
        Coefficients, w based on the original notations.

    feature_factors_ : 2d ndarray, shape [n_factors, n_features]
        Latent factors for all features. v based on the original
        notations. The learned factors can be viewed as the
        embeddings for each features. If a pair of features tends
        to co-occur often, then their embeddings should be
        close/similar (in terms of cosine similarity) to each other.

    history_ : list
        Loss function's history at each iteration, useful
        for evaluating whether the algorithm converged or not.

    References
    ----------
    .. [1] `S. Rendle Factorization Machines (2010)
            <http://www.csie.ntu.edu.tw/~b97053/paper/Rendle2010FM.pdf>`_ 
    """

    def __init__(self, n_iter = 10, n_factors = 10,
                 learning_rate = 0.1, reg_coef = 0.01,
                 reg_factors = 0.01, random_state = 1234, verbose = False):
        self.n_iter = n_iter
        self.verbose = verbose
        self.reg_coef = reg_coef
        self.n_factors = n_factors
        self.reg_factors = reg_factors
        self.random_state = random_state
        self.learning_rate = learning_rate

    def fit(self, X, y):
        """
        Fit the model to the input data and label.

        Parameters
        ----------
        X : scipy sparse csr_matrix, shape [n_samples, n_features]
            Data in sparse matrix format.

        y : 1d ndarray, shape [n_samples,]
            Training data's corresponding label.

        Returns
        -------
        self
        """

        n_samples, n_features = X.shape
        self.coef_ = np.zeros(n_features)
        self.intercept_ = 0.0

        # the factors are often initialized with a mean of 0 and standard deviation
        # of 1 / sqrt(number of latent factor specified)
        np.random.seed(self.random_state)
        self.feature_factors_ = np.random.normal(
            scale = 1 / np.sqrt(self.n_factors), size = (self.n_factors, n_features))
        
        # the gradient is implemented in a way that requires
        # the negative class to be labeled as -1 instead of 0
        y = y.copy().astype(np.int32)
        y[y == 0] = -1

        loop = range(self.n_iter)
        if self.verbose:
            loop = trange(self.n_iter)

        self.history_ = []
        for _ in loop:
            loss = _sgd_update(X.data, X.indptr, X.indices,
                               y, n_samples, n_features,
                               self.intercept_, self.coef_,
                               self.feature_factors_, self.n_factors,
                               self.learning_rate, self.reg_coef, self.reg_factors)
            self.history_.append(loss)

        return self

    def predict_proba(self, X):
        """
        Probability estimates. The returned estimates for
        all classes are ordered by the label of classes.

        Paramters
        ---------
        X : scipy sparse csr_matrix, shape [n_samples, n_features]
            Data in sparse matrix format.

        Returns
        -------
        proba : 2d ndarray, shape [n_samples, n_classes]
            The probability of the sample for each class in the model.
        """
        pred = self._predict(X)
        pred_proba = 1.0 / (1.0 + np.exp(-pred))
        proba = np.vstack((1 - pred_proba, pred_proba)).T
        return proba

    def _predict(self, X):
        """Similar to _predict_instance but vectorized for all samples"""
        linear_output = X * self.coef_
        v = self.feature_factors_.T
        term = (X * v) ** 2 - (X.power(2) * (v ** 2))
        factor_output = 0.5 * np.sum(term, axis = 1)
        return self.intercept_ + linear_output + factor_output

    def predict(self, X):
        """
        Predict class labels for samples in X.

        Parameters
        ----------
        X : scipy sparse csr_matrix, shape [n_samples, n_features]
            Data in sparse matrix format.

        Returns
        -------
        Predicted class label per sample.
        """
        pred_proba = self.predict_proba(X)[:, 1]
        return pred_proba.round().astype(np.int32)


@njit
def _sgd_update(data, indptr, indices, y, n_samples, n_features,
                w0, w, v, n_factors, learning_rate, reg_w, reg_v):
    """
    Compute the loss of the current iteration and update
    gradients accordingly.
    """
    loss = 0.0
    for i in range(n_samples):
        pred, summed = _predict_instance(data, indptr, indices, w0, w, v, n_factors, i)
        
        # calculate loss and its gradient
        loss += _log_loss(pred, y[i])
        loss_gradient = -y[i] / (np.exp(y[i] * pred) + 1.0)
    
        # update bias/intercept term
        w0 -= learning_rate * loss_gradient

        # update weight
        for index in range(indptr[i], indptr[i + 1]):
            feature = indices[index]
            w[feature] -= learning_rate * (loss_gradient * data[index] + 2 * reg_w * w[feature])

        # update factor
        for factor in range(n_factors):
            for index in range(indptr[i], indptr[i + 1]):
                feature = indices[index]
                term = summed[factor] - v[factor, feature] * data[index]
                v_gradient = loss_gradient * data[index] * term
                v[factor, feature] -= learning_rate * (v_gradient + 2 * reg_v * v[factor, feature])
    
    loss /= n_samples
    return loss


@njit
def _predict_instance(data, indptr, indices, w0, w, v, n_factors, i):
    """predicting a single instance"""
    summed = np.zeros(n_factors)
    summed_squared = np.zeros(n_factors)

    # linear output w * x
    pred = w0
    for index in range(indptr[i], indptr[i + 1]):
        feature = indices[index]
        pred += w[feature] * data[index]

    # factor output
    for factor in range(n_factors):
        for index in range(indptr[i], indptr[i + 1]):
            feature = indices[index]
            term = v[factor, feature] * data[index]
            summed[factor] += term
            summed_squared[factor] += term * term

        pred += 0.5 * (summed[factor] * summed[factor] - summed_squared[factor])
    
    # summed is the independent term that can be re-used
    # during the gradient update stage
    return pred, summed


@njit
def _log_loss(pred, y):
    """
    negative log likelihood of the
    current prediction and label, y.
    """
    return np.log(np.exp(-pred * y) + 1.0)

In [28]:
fm = FactorizationMachineClassifier(n_iter = 10, learning_rate = 0.01, n_factors=10, verbose=True)

In [29]:
fm.fit(X_train, y_train)

100%|██████████| 10/10 [05:24<00:00, 32.43s/it]


FactorizationMachineClassifier(learning_rate=0.01, verbose=True)

In [30]:
from sklearn.metrics import roc_auc_score
y_pred = fm.predict(X_train)

roc_auc_score(y_train, y_pred)

0.612640413285714